In [1]:
from IPython.display import display, HTML
display(HTML("<style>:root {--jp-notebook-max-width: 3000px;}</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

# 기계학습 (60점)

### 2023년도 NBA 농구 슛 시도 관련 데이터
data_path = './data/nba.csv'

NBA 각 경기에서 발생한 슛 시도 관련 데이터이다.     
자유튜(1점)은 없고 각 경기의 승패는 2,3점의 총 합으로 결정된다고 가정한다.

TEAM_NAME : 팀명     
PLAYER_ID : 슛을 쏜 선수    
POSITION_GROUP : 포지션    
GAME_ID : 게임명     
SHOT_MADE : 슛 성공여부 (True : 성공 ,False : 실패)     
ACTION_TYPE : 슛 종류     
SHOT_TYPE : 점수 (2점,3점)    
ZONE_ABB : 점수 존     
LOC_X : 슛을 쏜 x 좌표     
LOC_Y : 슛을 쏜 y 좌표      
QUARTER : 쿼터 (정규경기 4쿼터까지 쿼터당 12분)    
MINS_LEFT : 쿼터의 잔여 시간 (분)    
SECS_LEFT : 쿼터의 잔여 시간 (초)     

In [46]:
import pandas as pd
df =pd.read_csv('./data/nba.csv')
df.head()

,TEAM_NAME,PLAYER_ID,POSITION_GROUP,GAME_ID,SHOT_MADE,ACTION_TYPE,SHOT_TYPE,ZONE_ABB,LOC_X,LOC_Y,QUARTER,MINS_LEFT,SECS_LEFT
0,team_5,player_1,G,22300003,False,Driving Floating Jump Shot,2PT Field Goal,C,-0.4,17.45,1,11,1
1,team_5,player_2,F,22300003,True,Jump Shot,3PT Field Goal,C,1.5,30.55,1,10,26
2,team_5,player_3,G,22300003,True,Driving Layup Shot,2PT Field Goal,C,-3.3,6.55,1,9,46
3,team_5,player_1,G,22300003,True,Running Finger Roll Layup Shot,2PT Field Goal,C,-1.0,5.85,1,8,30
4,team_5,player_3,G,22300003,True,Cutting Layup Shot,2PT Field Goal,C,-0.0,6.25,1,8,8


### 기계학습1
아래의 기준으로 파생변수 3개를 만들어라 
- shot_distance (슛거리)     : LOC_X, LOX_Y 좌표의 제곱합에 루트를 씌운다
- net_play_time (경기경과시간) : 슛을 시도한 시점의 순 경기 시간을 초(sec)단위로 구한다. 쉬는시간은 고려하지 않는다. 4쿼터 종료시점(MINS_LEFT =0, SECS_LEFT=0)에 슛을 시도했다면 2880 값을 가져야 한다. 
- net_score : 해당 게임에 해당팀이 슛을 쏜 결과를 반영한 누적 득점

![imgs](./data/img/p1.png)

In [37]:
import numpy as np

In [61]:
df_copy = df.copy()
df_copy['shot_distance'] = (df_copy['LOC_X']**2 + df_copy['LOC_Y']**2)**(0.5)
df_copy['net_play_time'] = (df_copy['QUARTER'] - 1)*12*60 + (720 - df['MINS_LEFT']*60 - df['SECS_LEFT'])
df_copy['score'] =  df[df['SHOT_MADE'] == True]['SHOT_TYPE'].str[0].astype(int)
df_copy['score'].fillna(0, inplace = True)
df_copy['net_score'] = df_copy.groupby(['TEAM_NAME', 'GAME_ID'])[['score']].cumsum()
df_copy

,TEAM_NAME,PLAYER_ID,POSITION_GROUP,GAME_ID,SHOT_MADE,ACTION_TYPE,SHOT_TYPE,ZONE_ABB,LOC_X,LOC_Y,QUARTER,MINS_LEFT,SECS_LEFT,shot_distance,net_play_time,score,net_score
0,team_5,player_1,G,22300003,False,Driving Floating Jump Shot,2PT Field Goal,C,-0.4,17.45,1,11,1,17.454584,59,0.0,0.0
1,team_5,player_2,F,22300003,True,Jump Shot,3PT Field Goal,C,1.5,30.55,1,10,26,30.586803,94,3.0,3.0
2,team_5,player_3,G,22300003,True,Driving Layup Shot,2PT Field Goal,C,-3.3,6.55,1,9,46,7.334337,134,2.0,5.0
3,team_5,player_1,G,22300003,True,Running Finger Roll Layup Shot,2PT Field Goal,C,-1.0,5.85,1,8,30,5.934855,210,2.0,7.0
4,team_5,player_3,G,22300003,True,Cutting Layup Shot,2PT Field Goal,C,-0.0,6.25,1,8,8,6.250000,232,2.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207099,team_28,player_554,G,22301218,False,Jump Shot,3PT Field Goal,LC,11.9,29.85,4,0,48,32.134600,2832,0.0,90.0
207100,team_28,player_556,C,22301218,False,Tip Layup Shot,2PT Field Goal,C,1.6,14.25,4,0,45,14.339543,2835,0.0,90.0
207101,team_28,player_556,C,22301218,True,Jump Shot,2PT Field Goal,C,0.6,9.75,4,0,44,9.768444,2836,2.0,92.0
207102,team_28,player_552,G,22301218,True,Pullup Jump shot,3PT Field Goal,C,-1.7,34.55,4,0,37,34.591798,2843,3.0,95.0


In [59]:
df_copy.groupby(['TEAM_NAME', 'GAME_ID']).agg(np.cumsum)[['score']]

,score
0,0.0
1,3.0
2,5.0
3,7.0
4,9.0
...,...
207099,90.0
207100,90.0
207101,92.0
207102,95.0


### 기계학습2
데이터 EDA를 실시하라 (시각화 포함)

### 기계학습3     
3쿼터까지의 데이터를 바탕으로 승부를 예측하는 분류 모델을 만들려고 한다.
데이터를 아래 기준으로 전처리 진행한다.

- 각 행은 하나의 팀, 하나의 게임 아이디에 대한 정보를 나타낸다.
- 각 쿼터(1~3쿼터)를 6분간격으로 두 구간으로 나누어 새로운 파생변수를 만든다.      
    - 1_1_try (1쿼터의 0~6분)의 2점 실패 횟수)(1_1_try_2) , 3점 실패 횟수 (1_1_try_3)
    - 1_2_try (1쿼터의 6~12분)의 2점 실패 횟수 (1_1_try_2) ,3점 실패 횟수 (1_1_try_3)
    - ~ 3쿼터까지 총 6구간에 대해 12개의 컬럼
- 해당 게임의 팀별 1쿼터까지 득점, 3쿼터까지의 총 득점(2,3점) (총 2개컬럼, 1_q_score , 3_q_score)
- 4쿼터 후 최종 승패여부(result) (승 : 1 , 패 :0 ) - 자유투는 없다고 가정, 2,3점 만으로 게임의 승부를 결정, 승부가 결정나지 않은 게임은 없다

![img](./data/img/p2.png)


### 기계학습4     
game_id가 홀수로 끝나는 경우는 승리팀만, 짝수를 끝나는 경우는 패배팀만 필터한다.     
랜덤포레스트와 xgb를 사용하여 모델링을 진행하라. 데이터의 층화 추출을 통해 70%의 데이터로 학습하고 30% 데이터로 검증하라

### 캘리포니아 집값 데이터


MedInc (Median Income): 지역의 중간 소득. 가구의 중간 연간 소득을 나타냅니다.    
HouseAge (House Age): 지역의 평균 주택 연령. 주택이 지어진 지 얼마나 되었는지를 나타냅니다.     
AveRooms (Average Rooms): 지역의 평균 방 개수. 지역 내 집들의 평균 방 개수를 나타냅니다.     
AveBedrms (Average Bedrooms): 지역의 평균 침실 개수. 지역 내 집들의 평균 침실 개수를 나타냅니다.     
Population: 지역의 총 인구 수. 특정 지역 내 인구 수를 나타냅니다.     
AveOccup (Average Occupancy): 지역의 평균 거주 인원 수. 집당 평균 거주 인원 수를 나타냅니다.    
Latitude: 지역의 위도. 지구의 북쪽 또는 남쪽 위치를 나타내는 좌표입니다.       
Longitude: 지역의 경도. 지구의 동쪽 또는 서쪽 위치를 나타내는 좌표입니다.      
MedHouseVal (종속변수) : 집값 (천 달러 단위)

In [118]:
from sklearn.datasets import fetch_california_housing
cali = fetch_california_housing()
X,y=pd.DataFrame(cali.data,columns =cali['feature_names']), pd.DataFrame(cali.target,columns =['MedHouseVal'])
display(X.head())
display(y.head())

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


,MedHouseVal
0,4.526
1,3.585
2,3.521
3,3.413
4,3.422


### 기계학습5
데이터 EDA 및 시각화

### 기계학습6
변수들의 다중공선성을 검토하라

### 기계학습7     
회귀모델을 제작할 때 다중공선성을 줄이기 위한 방법 3가지를 서술하라

### 기계학습8     
훈련용 데이터와 평가용 데이터를 7:3으로 분리한 후, 다중선형회귀모델과 엘라스틱넷모델을 통해 회귀 모델링을 진행하고 rmse값을 기준으로 평가하라. 엘라스틱넷 모델의 경우, 정규화 강도 파라미터를 2가지, l1,l2 페널티를 3가지 방식으로 조정한 모델 총 6개를 만들고 결과에 대해 분석하라

# 통계 (40점)

### 통계1      
한 제조회사에서는 제품의 품질을 보장하기 위해 생산 라인에서 무작위로 10개의 제품을 샘플링하여 품질 검사를 실시한다. 이때, 제품이 불량일 확률은 23%다.

(1) 적어도 3개의 제품이 불량일 확률은?
(2) 2개 이하의 제품이 불량일 확률은?

### 통계2
어느 고객 서비스 센터에는 센터에는 평균적으로 1분에 3건의 전화가 걸려온다. 전화 빈도는 아래와 같은 조건을 가진다.
전화가 걸려오는 사건이 독립적이다.
특정 시간 동안에 전화가 걸려올 확률이 일정하다.
단위 시간 내에 동시에 두 개 이상의 전화가 걸려올 확률은 매우 낮다.

(1) 10분 동안 전화가 23통 이하로 걸려올 확률은?
(2) 10분 동안 전화가 정확히 30통 걸려올 확률은?

### 통계 3    
한 회사에서 직원들의 근무 만족도가 개선되었는지를 평가하기 위해, 직원들에게 개선 전과 개선 후의 만족도를 각각 설문조사했다. 만족도 점수는 1에서 10까지의 범위로 측정된다. 다음은 10명의 직원이 제공한 만족도 점수다

개선 전: [5, 6, 7, 5, 6, 8, 7, 5, 6, 9]      
개선 후: [6, 7, 8, 6, 7, 9, 8, 6, 7, 10]

### 통계 3-1
개선 전후의 만족도가 유의미하게 변화했는지 검정하려한다. 귀무가설과 대립가설을 설정하라

### 통계 3-2
통계검정을 실시하고 검정통계량과 pvalue값을 구하라. 결과를 분석하라

### 통계 4     
데이터셋 :./data/s5.csv

어느 부품공장의 1000개 부품 샘플의 무게(g)를 기록한 데이터셋이다.

In [124]:
df = pd.read_csv('./data/s5.csv')
df.head()

,w
0,78.8
1,72.0
2,74.9
3,81.2
4,79.3


이 데이터에서 183개의 샘플을 추출했을 때 샘플 무게의 평균이 69.5g에서 70.1g 사이에 있을 확률을 구하여라